In [ ]:
정규식 문법들
. => 아무글자 1개
캐릭터 => 그 글자 하나, 대소 문자 구분하기
?n => n이 있거나 없거나, *n, +n
{n,m} {0,} {1,}
^n, $n
[-] 범, [^] not
[작은코드-큰코드] => a-, ㄱ-ㅎ, 가-힣(11172자)
\d => 숫자, \w => 문자, \b => 단어경계, \s => 공백문자집합
\대문자 => 전부 반대 의미
() => 그룹 + 캡쳐링
(?:) => 그룹 + 논캡쳐링  # 캡쳐링? 
greedy vs lazy => +? 
나중에      처음에 일치  (주로 html 파싱할 때 쓴다.)

In [6]:
import re
data = '''
홍길동 980311-1231123
아무나 002131-2131454
누구  011112-3445211
'''
pat = re.compile( r'(\d{6})-\d{7}') # r'' 이게 뭐더라? # re.comple, re.sub란? 

print(re.sub( r'(\d{6})-\d{7}', r'\g<1>-*******',data))
print(re.sub( r'(\w+)\s+(\d{6})-\d{7}', r'\g<2> \g<1>',data))
print(re.sub( r'(\w+)\s+((\d{6})-\d{7})', r'\g<3>-******* \g<1>',data))



홍길동 980311-*******
아무나 002131-*******
누구  011112-*******


980311 홍길동
002131 아무나
011112 누구


980311-******* 홍길동
002131-******* 아무나
011112-******* 누구



In [17]:
import re
data = '''
naver.com => [.service]+.naver.com.kr.ac.edu.
www.naver.com
www.naver.com/index.html
http://www.naver.com
https://www.naver.com
https://mail.naver.com/latest/list?a=b
https://mail.naver.com/latest#top
ftp://ftp.daum.net
ftp://ftp:korea.ac.kr
'''

# scheme://service.(host)netloc/path/file?params... fragment
# re.search(r'[.]?[a-z0-9\-]{3,}[.][a-z0-9\-]{2,}+', data)
re.findall(r'(?:([a-z]+):\/\/)?([.]?[a-z0-9\-]{3,})((?:[.][a-z0-9\-]{2,})+)((?:\/[a-z]{3,})*)?(#\w+)?', data)

[('', 'naver', '.com', '', ''),
 ('', '.naver', '.com.kr.ac.edu', '', ''),
 ('', 'www', '.naver.com', '', ''),
 ('', 'www', '.naver.com', '/index', ''),
 ('http', 'www', '.naver.com', '', ''),
 ('https', 'www', '.naver.com', '', ''),
 ('https', 'mail', '.naver.com', '/latest/list', ''),
 ('https', 'mail', '.naver.com', '/latest', '#top'),
 ('ftp', 'ftp', '.daum.net', '', ''),
 ('', 'korea', '.ac.kr', '', '')]

In [22]:
data = '''
<article>
    <h1>제목</h1>
    <p>내용</p>
    <h2>제목2</h2>
    <h3>제목3<h4>제목4</h4></h3>  
</article>
'''
# 거지 같이 짜도 오류가 안난다.
# greedy면 </h3> 까지 인식
# lazy면 <h4> 까지 인식
# 위 두개의 다른 점을 알자.
# 위와 같은 문제가 자주 발생한다. 

# greedy
re.findall(r'<h[1-6]>(.+)</h[1-6]>', data)

['제목', '제목2', '제목3<h4>제목4</h4>']

In [23]:
# lazy
re.findall(r'<h[1-6]>(.+?)</h[1-6]>', data)

['제목', '제목2', '제목3<h4>제목4']

In [ ]:
다트게임 출시

다트를 3번 던져, 점수의 합계로 실력을 겨룬다.

점수 계산 로직 만들기
- 총 3번의 기회
- 각 기회 마다 얻을 수 있는 점수 0-10점
- 점수와 함께 Single(1승), Double(2승), Triple(3승) 영역으로 나뉜다. 
- 옵션으로 스타상(*)과 아차상(#)이 있다.
    - 스타상(*) 점수 2배 
    - 아차상(#) 점수 차감
- 첫번째 나온 스타상의 점수는 스타상점수 * 2
- 스타상 중첩 가능 그럼 원점수*(스타상*스타상)
- 스타상 아차상 중첩 가능 원점수*(스타상*-1)
- SDT는 점수 마다 하나씩 존재
- 스타상(*), 아차상(#)은 점수 마다 하나만 존재
- 1-10

In [25]:
bonus = {'S':1, 'D':2, 'T':3}
option = {'*':2, '#':-1}

def dart(s):
    score = list()

    if not re.search(r'(?:\d{2}[SDT][*#]?){3}',s):
        return '오류'
    
    re.search(r'(\d{1,2}[SDT][*#]?)(\d{1,2}[SDT][*#]?)(\d{1,2}[SDT][*#]?)',s)

    if not rst or len(rst.groups()) != 3: 
        return "오류"
    
    for g in rst.groups():
        rst = re.search(r'(\d{1,2})([SDT])([*#]?)'.groups())
        
        score.append(int(rst[0])**int(bonus[rst[1]])*(option[rst[2]] if rst[2] else 1))

        # 내 전에 나왔던 값
        if rst[2] == '*' and len(score) > 0:
            score[len(score)-1] *= option[rst[2]]
    
    print(score)
    return sum(score)

dart('1S2D*3T')

'오류'

In [7]:
import re
bonus = {'S':1, 'D':2, 'T':3}
option = {'*':2, '#':-1}

def dart(s):
    score = list()
    
    # 있어도 그만 없어도 그만
    if not re.search(r'(?:\d{1,2}[SDT][*#]?){3}', s):
        return '오류'
    
    rst = re.search(r'(\d{1,2}[SDT][*#]?)(\d{1,2}[SDT][*#]?)(\d{1,2}[SDT][*#]?)', s)

    if not rst or len(rst.groups()) != 3:
        return '오류'
    # 여기까지
    
    for g in rst.groups():
        print(re.search(r'([0-9]{1,2})([SDT])([*#])?', g))
        rst = re.search(r'(\d{1,2})([SDT])([*#])?', g).groups()
        
        score.append(int(rst[0])**int(bonus[rst[1]])*(option[rst[2]] if rst[2] else 1))
        
        print(score)
        if rst[2] == '*' and len(score) > 1:
            print(f'score[{len(score)-2}] *= option({rst[2]})')
            score[len(score)-2] *= option[rst[2]]
    print(score)
    return sum(score)
    
# dart('1S2D*3T')
# dart('1D2S#10S')
# dart('1D2S0T')
# dart('1S*2T*3S')
dart('1D#2S*3S')

<re.Match object; span=(0, 3), match='1D#'>
[-1]
<re.Match object; span=(0, 3), match='2S*'>
[-1, 4]
score[0] *= option(*)
<re.Match object; span=(0, 2), match='3S'>
[-2, 4, 3]
[-2, 4, 3]


5